<a href="https://colab.research.google.com/github/govind17/Information-Retrieval-Project/blob/main/LDA_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U Flask
!pip install git+https://github.com/boudinfl/pke.git
!pip install matplotlib
!python -m spacy download en_core_web_sm

In [2]:
# One time installation
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pritha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pritha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import numpy as np
import pandas as pd
import operator
import io
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import spacy

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

import flask
from flask import request
from flask_cors import CORS
from flask import Flask, Response
from keyPhrasification import key_phrasification

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

from io import BytesIO

import os
import joblib
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import ast
from keyPhrasification import key_phrasification

In [ ]:
# Indexing the cord-19 dataset
if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = 'C:/Users/Pritha/Desktop/SUBJECTS/PROJECT/Relevance feedback with XAI/Backend Code/Information-Retrieval-Project/terrier_cord19'
if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer
    indexer = pt.index.IterDictIndexer(pt_index_path)
    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(),
                              fields=('abstract',),
                              meta=('docno',))
else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
    index = pt.IndexFactory.of(index_ref)


In [ ]:
 # function to extract 5 words before and after a given word
def extract_context(text, keyword):
    # split the text into sentences using NLTK's sent_tokenize() function
    sentences = sent_tokenize(text)
    # iterate through each sentence
    context = []
    for sentence in sentences:
        # split the sentence into words using NLTK's word_tokenize() function
        words = word_tokenize(sentence)
        # check if the keyword is present in the sentence
        if keyword in words:
            # find the index of the keyword in the sentence
            keyword_index = words.index(keyword)
            # find the start and end indices for the context
            start_index = max(0, keyword_index - 5)
            end_index = min(len(words), keyword_index + 6)
            # extract the context and join the words together
            context.append(' '.join(words[start_index:end_index]))
    # join the contexts for each sentence together
    return ' '.join(context) if context else ''

def extract_context_from_keylist(keylist, abstract):
    result = []
    for key in keylist:
        result.append(extract_context(abstract, key))
    return result

In [ ]:
# Rocchio algorithm
def rocchio_algorithm(query_doc_vector, docs_relevant_vectors,
                      docs_irrelevant_vectors, key_relevant_vectors,
                      key_irrelevant_vectors,
                      alpha, beta, gamma, delta):

    # sum_of_rel_doc_vectors = docs_relevant_vectors.sum(axis=0)
    # print('sum_of_rel_doc_vectors', sum_of_rel_doc_vectors)
    sum_of_rel_doc_vectors=[]
    sum_of_irrel_doc_vectors=[]
    sum_of_rel_key_vectors=[]
    sum_of_irrel_key_vectors=[]
    for each_rel_doc_vector in docs_relevant_vectors:
        if len(sum_of_rel_doc_vectors) == 0:
            print('Length of sum_of_rel_doc_vectors is zero')
            sum_of_rel_doc_vectors = each_rel_doc_vector
        else:
            # print(each_rel_doc_vector)
            sum_of_rel_doc_vectors = list(map(operator.add, sum_of_rel_doc_vectors, each_rel_doc_vector))
            # print('SUM :', sum_of_rel_doc_vectors)
    for each_irrel_doc_vector in docs_irrelevant_vectors:
        if len(sum_of_irrel_doc_vectors) == 0:
            sum_of_irrel_doc_vectors = each_irrel_doc_vector
        else:
            sum_of_irrel_doc_vectors = list(map(operator.add, sum_of_irrel_doc_vectors, each_irrel_doc_vector))
    # sum_of_irrel_doc_vectors = docs_irrelevant_vectors.sum(axis=0)
    # print('sum_of_irrel_doc_vectors', sum_of_irrel_doc_vectors)
    for each_rel_key_vector in key_relevant_vectors:
        if len(sum_of_rel_key_vectors) == 0:
            sum_of_rel_key_vectors = each_rel_key_vector
        else:
            sum_of_rel_key_vectors = list(map(operator.add, sum_of_rel_key_vectors, each_rel_key_vector))
    # sum_of_rel_key_vectors = key_relevant_vectors.sum(axis=0)
    # print('sum_of_rel_key_vectors', sum_of_rel_key_vectors)
    for each_irrel_key_vector in key_irrelevant_vectors:
        if len(sum_of_irrel_key_vectors) == 0:
            sum_of_irrel_key_vectors = each_irrel_key_vector
        else:
            sum_of_irrel_key_vectors = list(map(operator.add, sum_of_irrel_key_vectors, each_irrel_key_vector))
    # sum_of_irrel_key_vectors = key_irrelevant_vectors.sum(axis=0)
    # print('sum_of_irrel_key_vectors', (delta/len(docs_relevant_vectors)) * np.array(sum_of_irrel_key_vectors))

    new_doc_vector_query = np.sum([np.array(query_doc_vector)
    , (alpha/len(docs_relevant_vectors)) * np.array(sum_of_rel_doc_vectors)
    , (beta/len(docs_irrelevant_vectors)) * np.array(sum_of_irrel_doc_vectors)
    , (gamma/len(docs_relevant_vectors)) * np.array(sum_of_rel_key_vectors)
    , (delta/len(docs_relevant_vectors)) * np.array(sum_of_irrel_key_vectors)], axis=0)
    df = pd.DataFrame({"a": [new_doc_vector_query]})
    return df.values

# Compute cosine scores
def compute_cosine_sim(new_query_vector, all_data, name):
    consine_similarities = []
    for index, data in all_data.iterrows():
        cosine_sim = cosine_similarity([np.array(new_query_vector[0])], [np.array(all_data[name][index])])
        consine_similarities.append(cosine_sim[0][0])
    return consine_similarities

# Rank data
def rank_data(new_query_vector, dataset, name):
    cosine_sim_values = compute_cosine_sim(new_query_vector, dataset, name)
    dataset['Cosine_Similarity_' + name] = cosine_sim_values
    dataset.drop(dataset[dataset['relevant'] == False], inplace=True)
    df_exploded = dataset.explode('KeyList')
    keyList_df = df_exploded.groupby('group')['KeyList'].apply(lambda x: ', '.join(x)).reset_index()
    relevant_KeyList = keyList_df['KeyList'].tolist()
    ps = PorterStemmer()
    dataset['Key_word_match_count'] = 0
    col = 'KeyList'
    for i, row in dataset.iterrows():
        words = [ps.stem(w) for w in nltk.word_tokenize(str(row[col])) if word_tokenize(str(row[col]).lower()) not in stopwords.words('english')]
        for word in relevant_KeyList:
            if ps.stem(word.str.lower()) in words:
                dataset.at[i, 'Key_word_match_count'] += 1
    dataset['feedback_score'] = dataset['Key_word_match_count']/5 + dataset['Cosine_Similarity_' + name]
    sorted_dataset = dataset.sort_values(by=['feedback_score'], ascending=False)
    return sorted_dataset



In [ ]:
#text preprocessing
def pre_processing_data(data):
    ps = PorterStemmer()
    tagged_dataset = [TaggedDocument(words=[ps.stem(w) for w in nltk.word_tokenize(str(_d)) if word_tokenize(str(_d).lower()) not in stopwords.words('english')], tags=[str(i)]) for i, _d in enumerate(data)]
    #tagged_dataset = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    return tagged_dataset

def model_doc2vec(model, tagged_data, num_epochs):
    model.build_vocab(tagged_data)
    model.train(tagged_data,total_examples=len(tagged_data), epochs=num_epochs)
    return model

def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, feature_vectors

def define_d2v_model(dataset):
    preprocessed_tagged_dataset = pre_processing_data(dataset)
    model_d2v = Doc2Vec(dm=1, vector_size=100, window = 10, negative=5, hs=0, min_count=2, sample = 0, alpha=0.025, min_alpha=0.001, dm_mean = 0, dbow_words=1)
    model = model_doc2vec(model_d2v, preprocessed_tagged_dataset, 100)
    return model

def get_vectors(input_data, model):
    preprocessed_tagged_input_data= pre_processing_data(input_data)
    id, vectors = np.array(vector_for_learning(model, preprocessed_tagged_input_data), dtype=object)
    return vectors

In [ ]:
monoT5 = MonoT5ReRanker(text_field='abstract')
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
cord19_df = pd.read_csv('C:/Users/Pritha/Desktop/SUBJECTS/PROJECT/Relevance feedback with XAI/Backend Code/Information-Retrieval-Project/cord19_sum.csv')
# model = define_d2v_model(cord19_df['abstract'])
model = joblib.load('gensim_model_2.pkl')
query_df = pd.DataFrame
reranked_df = pd.DataFrame

def search_query(query):
  index_ref2 = pt.IndexRef.of(pt_index_path + "/data.properties")
  index2 = pt.IndexFactory.of(index_ref2)
  # print(query)
  if not pt.started():
      pt.init()
  br = pt.BatchRetrieve(index2) % 50
  pipeline = (br >> pt.text.get_text(dataset, 'abstract') >> monoT5)
  search_result = pipeline.search(query)
  print(search_result)
  search_result = search_result.drop_duplicates(subset=['docno'])
  # search_result = search_result.drop_duplicates(subset=['summary'])

  filtered_docs = pd.merge(cord19_df, search_result, on = "docno", how = "inner")
  # filtered_docs = filtered_docs.drop_duplicates(subset=['docno', 'summary'])
  # return pd.read_csv('search_result.csv')
  return filtered_docs


# search_query('covid')

In [ ]:
def get_reranking(feedback_df : pd.DataFrame):
    global  query_df
    feedback_df["summary_vec"] = get_vectors(feedback_df['abstract'] + feedback_df['title'], model)
    feedback_df["keyList_vec"] = get_vectors(feedback_df['KeyList'], model)
    relevant_df = feedback_df.head(10).loc[feedback_df['relevant'] == True]
    irrelevant_df = feedback_df.head(10).loc[feedback_df['relevant'] == False]
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], relevant_df["summary_vec"].values, irrelevant_df["summary_vec"].values, relevant_df["keyList_vec"].values, irrelevant_df["keyList_vec"].values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, feedback_df, "keyList_vec")
    return sorted_dataset

def generate_plot():
    rel_vecs = []
    irrel_vecs = []
    # plt.canvas.flush_events()
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == True]['keyList_vec'].values):
        rel_vecs.append(data)
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == False]['keyList_vec'].values):
        irrel_vecs.append(data)
    pca = PCA(n_components=2)
    pca_rel = pca.fit_transform(rel_vecs)
    pca_irrel = pca.fit_transform(irrel_vecs)
    query =np.concatenate(([np.array(query_df.iloc[0]['query_vec'])], [np.array(query_df.iloc[0]['new_query_vec'])]), axis=0)
    pca_query = pca.fit_transform(query)
    print("Query vector for matplotlib 1: ", pca_query[0][0], pca_query[0][1])
    print("Query vector for matplotlib 2: ", pca_query[1][0], pca_query[1][1])
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    axis.scatter(pca_rel[:,0], pca_rel[:,1], marker="8", label="relevant")
    axis.scatter(pca_irrel[:,0], pca_irrel[:,1], marker="*", label="irrelevant")
    axis.scatter(pca_query[0][0], pca_query[0][1], marker="s", label="old_query")
    axis.scatter(pca_query[1][0], pca_query[1][1], marker="s", label="new_query")
    axis.legend()

    return fig

In [ ]:
# function to extract 5 words before and after a given word
def extract_context(text, keyword):
    # split the text into sentences using NLTK's sent_tokenize() function
    sentences = sent_tokenize(text)
    # iterate through each sentence
    context = []
    for sentence in sentences:
        # split the sentence into words using NLTK's word_tokenize() function
        words = word_tokenize(sentence)
        # check if the keyword is present in the sentence
        if keyword in words:
            # find the index of the keyword in the sentence
            keyword_index = words.index(keyword)
            # find the start and end indices for the context
            start_index = max(0, keyword_index - 5)
            end_index = min(len(words), keyword_index + 6)
            # extract the context and join the words together
            context.append(' '.join(words[start_index:end_index]))
    # join the contexts for each sentence together
    return ' '.join(context) if context else ''

def extract_context_from_keylist(keylist, abstract):
    result = []
    for key in keylist:
        result.append(extract_context(abstract, key))
    return result

In [ ]:
# cord19_df = pd.read_csv('/Users/GovindShukla/Downloads/cord19_sum_key.csv')
#
# model = define_d2v_model(cord19_df[~cord19_df['abstract'].isna()]['abstract'])

In [ ]:
#
# # Save the model as a pickle in a file
# joblib.dump(model, 'gensim_model_2.pkl')
# model = joblib.load('/Users/GovindShukla/Desktop/Information-Retrieval-Project/gensim_model_2.pkl')

In [ ]:
import base64
app = flask.Flask(__name__)

# app.config["DEBUG"] = True
CORS(app)
from flask import send_file
@app.route('/query', methods=['GET'])
def search():
    query = request.args.get('searchString')
    print(query)
    searchResults = search_query(query)
    print(searchResults.head())
    searchResultswithkeys = key_phrasification(searchResults)
    # print(searchResultswithkeys)

    # for index, row in searchResultswithkeys.iterrows():
    #     # print('Type of record: ', type(row['KeyList']))
    #     # keylist = ast.literal_eval(row['KeyList'])
    #     keylist = row['KeyList']
    #     searchResultswithkeys.loc[index, 'context_key_1'] = extract_context(row['abstract'] + row['title'], keylist[0])
    #     searchResultswithkeys.loc[index, 'context_key_2'] = extract_context(row['abstract'] + row['title'], keylist[1])
    #     searchResultswithkeys.loc[index, 'context_key_3'] = extract_context(row['abstract'] + row['title'], keylist[2])
    #     searchResultswithkeys.loc[index, 'context_key_4'] = extract_context(row['abstract'] + row['title'], keylist[3])
    #     searchResultswithkeys.loc[index, 'context_key_5'] = extract_context(row['abstract'] + row['title'], keylist[4])
    # ps = PorterStemmer()

    # searchResultswithkeys['query_count'] = 0
    # columns_to_search = ['context_key_1', 'context_key_2', 'context_key_3', 'context_key_4', 'context_key_5']
    # for i, row in searchResultswithkeys[columns_to_search].iterrows():
    #     for col in searchResultswithkeys[columns_to_search].columns:
    #         words = [ps.stem(w) for w in nltk.word_tokenize(str(row[col])) if word_tokenize(str(row[col]).lower()) not in stopwords.words('english')]
    #         for word in query.lower().split():
    #             if ps.stem(word) in words:
    #                 searchResultswithkeys.at[i, 'query_count'] += 1
    global query_df
    query_dict = {'query': [query], 'query_vec' : [np.nan], 'new_query_vec' : [np.nan] }
    query_df = pd.DataFrame(data=query_dict, index=[0])
    # print(searchResultswithkeys)
    # searchResultswithkeys.to_csv('search_result.csv')
    return searchResultswithkeys.to_json(orient='records')

# Feedback
@app.route('/feedback', methods=['POST'])
def fetchFeedback():
    global reranked_df
    feedbackJson = request.json['updates']
    relevanceList = []
    if len(feedbackJson):
        for doc in feedbackJson:
            for relevance in doc['value']:
                relevanceList.append(relevance)
    feedback_df = pd.DataFrame(relevanceList)
    # feedback_df.drop(columns=['bntStyle'], inplace=True)
    reranked_df = get_reranking(feedback_df)
    print(reranked_df)
    return reranked_df.to_json(orient='records')

@app.route('/plot')
def plot():
    # Generate the plot


    fig = generate_plot()
    canvas = FigureCanvas(fig)
    output = io.BytesIO()
    canvas.print_png(output)
    response = Response(output.getvalue(), mimetype='image/png')

    return response
    # buffer.seek(0)

    # image_png = buffer.getvalue()
    # buffer.close()
    # graphic = base64.b64encode(image_png).decode('utf-8')
    # graphic = graphic.decode('utf-8')
    # fig.savefig('plot.png')

    # return send_file('plot.png', mimetype='image/png')

if __name__ == "__main__":

    app.run(host='0.0.0.0',port=5149)

In [16]:
import base64
app = flask.Flask(__name__)

# app.config["DEBUG"] = True
CORS(app)
from flask import send_file
@app.route('/query', methods=['GET'])
def search():
    query = request.args.get('searchString')
    print(query)
    searchResults = search_query(query)
    print(searchResults.head())
    searchResultswithkeys = key_phrasification(searchResults)
    # print(searchResultswithkeys)

    for index, row in searchResultswithkeys.iterrows():
        # print('Type of record: ', type(row['KeyList']))
        # keylist = ast.literal_eval(row['KeyList'])
        keylist = row['KeyList']
        searchResultswithkeys.loc[index, 'context_key_1'] = extract_context(row['abstract'] + row['title'], keylist[0])
        searchResultswithkeys.loc[index, 'context_key_2'] = extract_context(row['abstract'] + row['title'], keylist[1])
        searchResultswithkeys.loc[index, 'context_key_3'] = extract_context(row['abstract'] + row['title'], keylist[2])
        searchResultswithkeys.loc[index, 'context_key_4'] = extract_context(row['abstract'] + row['title'], keylist[3])
        searchResultswithkeys.loc[index, 'context_key_5'] = extract_context(row['abstract'] + row['title'], keylist[4])
    ps = PorterStemmer()
    # searchResultswithkeys = key_phrasification(searchResults)
    searchResultswithkeys['query_count'] = 0
    columns_to_search = ['context_key_1', 'context_key_2', 'context_key_3', 'context_key_4', 'context_key_5']
    for i, row in searchResultswithkeys[columns_to_search].iterrows():
        for col in searchResultswithkeys[columns_to_search].columns:
            words = [ps.stem(w) for w in nltk.word_tokenize(str(row[col])) if word_tokenize(str(row[col]).lower()) not in stopwords.words('english')]
            for word in query.lower().split():
                if ps.stem(word) in words:
                    searchResultswithkeys.at[i, 'query_count'] += 1
    global query_df
    query_dict = {'query': [query], 'query_vec' : [np.nan], 'new_query_vec' : [np.nan] }
    query_df = pd.DataFrame(data=query_dict, index=[0])
    # print(searchResultswithkeys)
    # searchResultswithkeys.to_csv('search_result.csv')
    return searchResultswithkeys.to_json(orient='records')

# Feedback
@app.route('/feedback', methods=['POST'])
def fetchFeedback():
    global reranked_df
    feedbackJson = request.json['updates']
    relevanceList = []
    if len(feedbackJson):
        for doc in feedbackJson:
            for relevance in doc['value']:
                relevanceList.append(relevance)
    feedback_df = pd.DataFrame(relevanceList)
    # feedback_df.drop(columns=['bntStyle'], inplace=True)
    reranked_df = get_reranking(feedback_df)
    print(reranked_df)
    return reranked_df.to_json(orient='records')

@app.route('/plot')
def plot():
    # Generate the plot


    fig = generate_plot()
    canvas = FigureCanvas(fig)
    output = io.BytesIO()
    canvas.print_png(output)
    response = Response(output.getvalue(), mimetype='image/png')

    return response
    # buffer.seek(0)

    # image_png = buffer.getvalue()
    # buffer.close()
    # graphic = base64.b64encode(image_png).decode('utf-8')
    # graphic = graphic.decode('utf-8')
    # fig.savefig('plot.png')

    # return send_file('plot.png', mimetype='image/png')

if __name__ == "__main__":

    app.run(host='0.0.0.0',port=5149)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5149
 * Running on http://192.168.0.15:5149
Press CTRL+C to quit


pandemic


monoT5: 100%|██████████| 13/13 [01:42<00:00,  7.91s/batches]


   qid   docid     docno     query  \
45   1   16141  w76osch5  pandemic   
48   1   87121  0usvjfos  pandemic   
30   1   82682  4igvc039  pandemic   
29   1  137061  fhp5f78b  pandemic   
22   1  159665  mb17dnf2  pandemic   
43   1  136292  h1rohel3  pandemic   
42   1    6267  kfbu10oy  pandemic   
16   1  182439  eruilkrq  pandemic   
15   1   92504  rjdb1qrt  pandemic   
4    1   52415  8n0mwad5  pandemic   
31   1   80058  5go8mo4m  pandemic   
23   1   14743  suzq01ti  pandemic   
21   1  130342  eo9xel0n  pandemic   
10   1  161087  9s7y8guc  pandemic   
32   1   82436  6s1zxghw  pandemic   
46   1    6321  si9ty2dt  pandemic   
14   1  134190  p94jups3  pandemic   
34   1  107501  nj3nkh3n  pandemic   
0    1     513  dzzlpz4o  pandemic   
35   1  165342  52z9v472  pandemic   
33   1   96508  eioez25e  pandemic   
37   1   25664  9ent6mpa  pandemic   
18   1    5310  k5y4x076  pandemic   
12   1   61346  v932f5ni  pandemic   
47   1  133830  3smc4ye3  pandemic   
28   1  1503

...

127.0.0.1 - - [08/Apr/2023 13:18:49] "GET /query?searchString=pandemic HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2023 13:19:12] "OPTIONS /feedback HTTP/1.1" 200 -


Length of sum_of_rel_doc_vectors is zero
       docno                                              title  \
31  3smc4ye3  US school/academic institution disaster and pa...   
9   ls6on0sg  A perspective on the 2009 A/H1N1 influenza pan...   
8   9ent6mpa                               Models for Influenza   
22  mgsgfi77  Epidemiology of pandemic influenza: use of sur...   
1   v5y9s97b  Spatial and Temporal Characteristics of the 20...   
23  0usvjfos  Preparing for the next influenza pandemic: a N...   
49  08gqn86z  Influenza pandemic preparedness: motivation fo...   
25  rjdb1qrt                Understanding dynamics of pandemics   
34  zs8tejm5  From SARS in 2003 to H1N1 in 2009: lessons lea...   
38  nada82w4  Non-pharmaceutical public health interventions...   
36  2sw82osi  Global Nationalism in Times of the COVID-19 Pa...   
14  v932f5ni  The use of mathematical models to inform influ...   
0   dzzlpz4o  We should not be complacent about our populati...   
46  eruilkrq         

127.0.0.1 - - [08/Apr/2023 13:20:05] "POST /feedback HTTP/1.1" 200 -


Query vector for matplotlib 1:  -21.121517 4.412226e-06
Query vector for matplotlib 2:  21.121523 4.4122244e-06


127.0.0.1 - - [08/Apr/2023 13:20:43] "GET /plot HTTP/1.1" 200 -


In [ ]:
def get_reranking(feedback_df : pd.DataFrame, model):
    global  query_df
    feedback_df["summary_vec"] = get_vectors(feedback_df['summary'], model)
    feedback_df["keyList_vec"] = get_vectors(feedback_df['KeyList'], model)
    relevant_df = feedback_df.loc[feedback_df['relevant'] == True]
    irrelevant_df = feedback_df.loc[feedback_df['relevant'] == False]
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], relevant_df["summary_vec"].values, irrelevant_df["summary_vec"].values, relevant_df["keyList_vec"].values, irrelevant_df["keyList_vec"].values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, feedback_df, "keyList_vec")
    feedback_df.drop(feedback_df[feedback_df['relevant'] == False], inplace=True)
    df_exploded = feedback_df.explode('KeyList')
    keyList_df = df_exploded.groupby('group')['KeyList'].apply(lambda x: ', '.join(x)).reset_index()
    relevant_KeyList = keyList_df['KeyList'].tolist()

    return sorted_dataset

In [ ]:
def get_reranking(feedback_df : pd.DataFrame, model):
    global  query_df
    feedback_df["summary_vec"] = get_vectors(feedback_df['summary'], model)
    feedback_df["keyList_vec"] = get_vectors(feedback_df['KeyList'], model)
    relevant_df = feedback_df.loc[feedback_df['relevant'] == True]
    irrelevant_df = feedback_df.loc[feedback_df['relevant'] == False]
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], relevant_df["summary_vec"].values, irrelevant_df["summary_vec"].values, relevant_df["keyList_vec"].values, irrelevant_df["keyList_vec"].values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, feedback_df, "keyList_vec")
    return sorted_dataset

In [ ]:
def cal_RM():
    data = pd.read_csv('/Users/GovindShukla/Desktop/Information-Retrieval-Project/search_result.csv')
    print(data['summary'])
    model = define_d2v_model(data['abstract'])
    q = {'query': ['Covid19'], 'query_vec' : [np.nan], 'new_query_vec' : [np.nan] }
    query_df = pd.DataFrame(data=q, index=[0])
    data["summary_vec"] = get_vectors(data['summary'], model)
    data["keyList_vec"] = get_vectors(data['KeyList'], model)
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], data["summary_vec"].head(3).values, data["summary_vec"].head(7).values,
                      data["keyList_vec"].head(3).values, data["keyList_vec"].head(7).values, 1.0, 0.5,1.0, 0.5)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, data, "keyList_vec")
    print(sorted_dataset['summary'])
cal_RM()

In [ ]:
rel_docs = []
irrel_docs = []
for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == True]['summary_vec'].values):
    rel_docs.append(data)
for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == False]['summary_vec'].values):
    irrel_docs.append(data)


In [ ]:
reranked_df.loc[reranked_df['relevant'] == False]

In [ ]:
reranked_df.head(3)

In [ ]:
def generate_plot():
    rel_docs = []
    irrel_docs = []
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == True]['summary_vec'].values):
        rel_docs.append(data)
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == False]['summary_vec'].values):
        irrel_docs.append(data)
    pca = PCA(n_components=2)
    pca_rel = pca.fit_transform(rel_docs)
    pca_irrel = pca.fit_transform(irrel_docs)
    print(query_df)
    query =np.concatenate(([np.array(query_df.iloc[0]['query_vec'])], [np.array(query_df.iloc[0]['new_query_vec'])]), axis=0)
    pca_query = pca.fit_transform(query)

    plt.scatter(pca_rel[:,0], pca_rel[:,1], marker="8", label="rel_doc")
    plt.scatter(pca_irrel[:,0], pca_irrel[:,1], marker="*", label="irrel_doc")
    plt.scatter(pca_query[0][0], pca_query[0][1], marker="s", label="old_query")
    plt.scatter(pca_query[1][0], pca_query[1][1], marker="s", label="new_query")
    plt.legend()

    return plt

generate_plot()
